In [1]:
import pandas as pd

In [13]:
sales_cleaned = pd.read_csv("data/sales_cleaned.csv").drop(columns=['Unnamed: 0'], axis = 1)
tickets = pd.read_csv("data/dataset3.csv")
logs_joined = pd.read_csv("data/logs_joined.csv")
logs = logs_joined[["customer_id", "date", "logins", "feature_events", "session_minutes"]]

In [14]:
sales_cleaned.isna().sum()

customer_id                    0
company_name                   0
country                        0
region                         0
is_eu                          0
industry                       0
company_size_bucket            0
annual_contract_value          0
product_tier                   0
sales_segment                  0
acquisition_channel            0
contract_start_date            0
contract_end_date           1428
renewed_flag                   0
discount_pct                   0
initial_onboarding_score       0
is_churned                     0
dtype: int64

In [15]:
tickets.isna().sum()

ticket_id               0
customer_id             0
created_at              0
channel                 0
priority                0
issue_category          0
ticket_text             0
sentiment               0
first_response_hours    0
resolution_hours        0
resolved                0
dtype: int64

In [16]:
logs.isna().sum()

customer_id        0
date               0
logins             0
feature_events     0
session_minutes    0
dtype: int64

In [17]:
print(f'Sales Columns: {sales_cleaned.columns}')
print(f'Tickets Columns: {tickets.columns}')
print(f'Logs Columns: {logs.columns}')

Sales Columns: Index(['customer_id', 'company_name', 'country', 'region', 'is_eu', 'industry',
       'company_size_bucket', 'annual_contract_value', 'product_tier',
       'sales_segment', 'acquisition_channel', 'contract_start_date',
       'contract_end_date', 'renewed_flag', 'discount_pct',
       'initial_onboarding_score', 'is_churned'],
      dtype='object')
Tickets Columns: Index(['ticket_id', 'customer_id', 'created_at', 'channel', 'priority',
       'issue_category', 'ticket_text', 'sentiment', 'first_response_hours',
       'resolution_hours', 'resolved'],
      dtype='object')
Logs Columns: Index(['customer_id', 'date', 'logins', 'feature_events', 'session_minutes'], dtype='object')


### Get Periods

In [22]:
# 2. Define the Q3 Time Periods
Q3_START = pd.Timestamp("2024-07-01")

def get_period(date):
    if pd.isna(date): return "No Date"
    dt = pd.to_datetime(date)
    if dt < Q3_START:
        return "Before"
    else:
        return "After"
    
# 3. Create 'period' columns (using .loc to avoid SettingWithCopyWarning)
logs.loc[:, 'date'] = pd.to_datetime(logs['date'])
logs.loc[:, 'period'] = logs['date'].apply(get_period)

tickets.loc[:, 'created_at'] = pd.to_datetime(tickets['created_at'])
tickets.loc[:, 'period'] = tickets['created_at'].apply(get_period)

### OHE Tickets

In [35]:
# 1. Create dummy variables for the categorical columns
# (We exclude ticket_text as it is unstructured)
ticket_dummies = pd.get_dummies(tickets[['customer_id', 'period', 'channel', 'priority', 'issue_category', 'ticket_text']], 
                                columns=['channel', 'priority', 'issue_category', 'ticket_text'])

ticket_dummies.head()

,customer_id,period,channel_Chat,channel_Email,channel_In-App,channel_Phone,priority_High,priority_Low,priority_Medium,priority_Urgent,...,ticket_text_Our discount was not applied to this renewal.,ticket_text_Pricing discussed in the demo does not match our invoice.,ticket_text_Sales promised a custom report that doesn't exist.,ticket_text_Search results are not clearly sorted.,ticket_text_The new dashboard is confusing for my staff.,ticket_text_The reporting page keeps timing out.,ticket_text_VAT appears to be calculated incorrectly.,ticket_text_We expected multi-warehouse support out of the box.,ticket_text_We were charged for extra users we don't have.,"ticket_text_We were told onboarding would be fully managed, but it wasn't."
0,C100000,Before,True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,C100001,Before,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,C100002,After,False,True,False,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,False
3,C100003,After,False,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,C100004,After,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [36]:
# 2. Aggregate these dummies by summing them up
# This tells us HOW MANY of each type the customer had in that period
ticket_features = ticket_dummies.groupby(['customer_id', 'period']).sum().reset_index()
ticket_features.head()

,customer_id,period,channel_Chat,channel_Email,channel_In-App,channel_Phone,priority_High,priority_Low,priority_Medium,priority_Urgent,...,ticket_text_Our discount was not applied to this renewal.,ticket_text_Pricing discussed in the demo does not match our invoice.,ticket_text_Sales promised a custom report that doesn't exist.,ticket_text_Search results are not clearly sorted.,ticket_text_The new dashboard is confusing for my staff.,ticket_text_The reporting page keeps timing out.,ticket_text_VAT appears to be calculated incorrectly.,ticket_text_We expected multi-warehouse support out of the box.,ticket_text_We were charged for extra users we don't have.,"ticket_text_We were told onboarding would be fully managed, but it wasn't."
0,C100000,Before,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,C100001,Before,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,C100002,After,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,C100003,After,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,C100004,After,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# 3. Add the numeric averages back in (Sentiment, Resolution Time)
ticket_numeric = tickets.groupby(['customer_id', 'period']).agg({
    'sentiment': 'mean',
    'resolution_hours': 'mean',
    'first_response_hours': 'mean', 
    'resolution_hours': "mean", 
    'resolved': "mean",
    'ticket_id': 'count'
}).rename(columns={'ticket_id': 'total_ticket_count'}).reset_index()
ticket_numeric.head()

,customer_id,period,sentiment,resolution_hours,first_response_hours,resolved,total_ticket_count
0,C100000,Before,0.44,14.88,0.11,1.0,1
1,C100001,Before,-0.50,9.37,8.12,1.0,1
2,C100002,After,-0.50,31.65,2.97,1.0,1
3,C100003,After,-0.11,12.49,0.10,1.0,1
4,C100004,After,0.20,35.67,5.19,1.0,1


In [40]:
tickets.head(1)

,ticket_id,customer_id,created_at,channel,priority,issue_category,ticket_text,sentiment,first_response_hours,resolution_hours,resolved,period
0,200001,C100000,2024-05-12 23:17:00,Chat,Medium,product_performance,Dashboard loads very slowly during peak hours.,0.44,0.11,14.88,1,Before


In [39]:
# 4. Merge them together
tickets_flattened = pd.merge(ticket_features, ticket_numeric, on=['customer_id', 'period'], how='inner')
tickets_flattened.head()

,customer_id,period,channel_Chat,channel_Email,channel_In-App,channel_Phone,priority_High,priority_Low,priority_Medium,priority_Urgent,...,ticket_text_The reporting page keeps timing out.,ticket_text_VAT appears to be calculated incorrectly.,ticket_text_We expected multi-warehouse support out of the box.,ticket_text_We were charged for extra users we don't have.,"ticket_text_We were told onboarding would be fully managed, but it wasn't.",sentiment,resolution_hours,first_response_hours,resolved,total_ticket_count
0,C100000,Before,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0.44,14.88,0.11,1.0,1
1,C100001,Before,0,1,0,0,0,1,0,0,...,0,0,0,0,0,-0.50,9.37,8.12,1.0,1
2,C100002,After,0,1,0,0,0,0,1,0,...,0,0,0,0,0,-0.50,31.65,2.97,1.0,1
3,C100003,After,0,0,0,1,1,0,0,0,...,0,0,0,0,0,-0.11,12.49,0.10,1.0,1
4,C100004,After,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0.20,35.67,5.19,1.0,1


In [43]:
logs_grouped = logs.groupby(["customer_id", "period"]).mean().reset_index().drop(columns=["date"], axis = 1)
logs_grouped.head()

,customer_id,period,logins,feature_events,session_minutes
0,C100000,After,4.152174,1.548913,27.326420
1,C100000,Before,3.818681,1.318681,26.907198
2,C100001,After,0.978261,0.255435,12.655226
3,C100001,Before,0.972527,0.340659,13.701538
4,C100002,After,0.000000,0.000000,0.000000


### Joining

In [45]:
# Step 1: Combine the two temporal (time-based) tables first
# We use an outer join on both customer_id AND period.
# This ensures we capture every period a customer was active in either system.
activity_combined = pd.merge(
    logs_grouped, 
    tickets_flattened, 
    on=['customer_id', 'period'], 
    how='outer'
)
print(len(activity_combined))
activity_combined.head()

6000


,customer_id,period,logins,feature_events,session_minutes,channel_Chat,channel_Email,channel_In-App,channel_Phone,priority_High,...,ticket_text_The reporting page keeps timing out.,ticket_text_VAT appears to be calculated incorrectly.,ticket_text_We expected multi-warehouse support out of the box.,ticket_text_We were charged for extra users we don't have.,"ticket_text_We were told onboarding would be fully managed, but it wasn't.",sentiment,resolution_hours,first_response_hours,resolved,total_ticket_count
0,C100000,After,4.152174,1.548913,27.326420,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C100000,Before,3.818681,1.318681,26.907198,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.44,14.88,0.11,1.0,1.0
2,C100001,After,0.978261,0.255435,12.655226,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C100001,Before,0.972527,0.340659,13.701538,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.50,9.37,8.12,1.0,1.0
4,C100002,After,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.50,31.65,2.97,1.0,1.0


In [54]:
# Step 2: Join the combined activity with the static Sales data
# We join on customer_id only because sales data applies to all periods.
# Using 'outer' here ensures that "Quiet Customers" (Sales only) 
# and "Shadow Users" (Logs only) are both preserved.
master_giant = pd.merge(
    sales_cleaned, 
    activity_combined, 
    on='customer_id', 
    how='outer'
)
print(len(master_giant))
master_giant.head()

6000


,customer_id,company_name,country,region,is_eu,industry,company_size_bucket,annual_contract_value,product_tier,sales_segment,...,ticket_text_The reporting page keeps timing out.,ticket_text_VAT appears to be calculated incorrectly.,ticket_text_We expected multi-warehouse support out of the box.,ticket_text_We were charged for extra users we don't have.,"ticket_text_We were told onboarding would be fully managed, but it wasn't.",sentiment,resolution_hours,first_response_hours,resolved,total_ticket_count
0,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,...,0.0,0.0,0.0,0.0,0.0,0.44,14.88,0.11,1.0,1.0
2,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,...,0.0,0.0,0.0,0.0,0.0,-0.50,9.37,8.12,1.0,1.0
4,C100002,Company_2,US,North America,0,Manufacturing,1-10,1770.83,Starter,SMB_Field,...,0.0,0.0,0.0,0.0,0.0,-0.50,31.65,2.97,1.0,1.0


In [55]:
# Step 3: Fill Nulls for Count/Sum columns
# After an outer join, any customer without activity in a period will have NaN.
# These should logically be 0.
count_cols = [col for col in master_giant.columns if 'count' in col or 'logins' in col or 'events' in col]
master_giant[count_cols] = master_giant[count_cols].fillna(0)
print(len(master_giant))
master_giant.head()

6000


,customer_id,company_name,country,region,is_eu,industry,company_size_bucket,annual_contract_value,product_tier,sales_segment,...,ticket_text_The reporting page keeps timing out.,ticket_text_VAT appears to be calculated incorrectly.,ticket_text_We expected multi-warehouse support out of the box.,ticket_text_We were charged for extra users we don't have.,"ticket_text_We were told onboarding would be fully managed, but it wasn't.",sentiment,resolution_hours,first_response_hours,resolved,total_ticket_count
0,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,...,0.0,0.0,0.0,0.0,0.0,0.44,14.88,0.11,1.0,1.0
2,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,...,0.0,0.0,0.0,0.0,0.0,-0.50,9.37,8.12,1.0,1.0
4,C100002,Company_2,US,North America,0,Manufacturing,1-10,1770.83,Starter,SMB_Field,...,0.0,0.0,0.0,0.0,0.0,-0.50,31.65,2.97,1.0,1.0


In [56]:
# fill NA in null values
# 1. Identify all columns created by One-Hot Encoding
# (Usually they contain a prefix like 'channel_' or 'priority_')
ohe_cols = [col for col in master_giant.columns if '_' in col and 
            any(pref in col for pref in ['channel', 'priority', 'issue_category', 'ticket_text'])]

# 2. Fill them with 0
master_giant[ohe_cols] = master_giant[ohe_cols].fillna(0)

# 3. Also fill your main count columns
# master_giant['resolution_hours'] = master_giant['resolution_hours'].fillna(0)
# master_giant['first_response_hours'] = master_giant['first_response_hours'].fillna(0)
master_giant['resolved'] = master_giant['resolved'].fillna(0)
master_giant['total_ticket_count'] = master_giant['total_ticket_count'].fillna(0)

In [57]:
master_giant.head()

,customer_id,company_name,country,region,is_eu,industry,company_size_bucket,annual_contract_value,product_tier,sales_segment,...,ticket_text_The reporting page keeps timing out.,ticket_text_VAT appears to be calculated incorrectly.,ticket_text_We expected multi-warehouse support out of the box.,ticket_text_We were charged for extra users we don't have.,"ticket_text_We were told onboarding would be fully managed, but it wasn't.",sentiment,resolution_hours,first_response_hours,resolved,total_ticket_count
0,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0
1,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,...,0.0,0.0,0.0,0.0,0.0,0.44,14.88,0.11,1.0,1.0
2,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0
3,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,...,0.0,0.0,0.0,0.0,0.0,-0.50,9.37,8.12,1.0,1.0
4,C100002,Company_2,US,North America,0,Manufacturing,1-10,1770.83,Starter,SMB_Field,...,0.0,0.0,0.0,0.0,0.0,-0.50,31.65,2.97,1.0,1.0


In [58]:
master_giant.columns

Index(['customer_id', 'company_name', 'country', 'region', 'is_eu', 'industry',
       'company_size_bucket', 'annual_contract_value', 'product_tier',
       'sales_segment', 'acquisition_channel', 'contract_start_date',
       'contract_end_date', 'renewed_flag', 'discount_pct',
       'initial_onboarding_score', 'is_churned', 'period', 'logins',
       'feature_events', 'session_minutes', 'channel_Chat', 'channel_Email',
       'channel_In-App', 'channel_Phone', 'priority_High', 'priority_Low',
       'priority_Medium', 'priority_Urgent', 'issue_category_billing_admin',
       'issue_category_product_performance',
       'issue_category_product_usability', 'issue_category_sales_expectation',
       'ticket_text_Bulk upload flow is not intuitive.',
       'ticket_text_Dashboard loads very slowly during peak hours.',
       'ticket_text_I need to change our billing email and can't find the option.',
       'ticket_text_Inventory sync takes too long to complete.',
       'ticket_text_It

In [59]:
master_giant.to_csv('data/master_giant.csv', index=False)

In [60]:
master_giant["period"].unique()

array(['After', 'Before'], dtype=object)

In [61]:
master_before = master_giant[master_giant["period"] == "Before"]
master_after = master_giant[master_giant["period"] == "After"]

In [62]:
master_before.to_csv('data/master_before.csv', index=False)
master_after.to_csv('data/master_after.csv', index=False)

In [63]:
sales_cleaned

,customer_id,company_name,country,region,is_eu,industry,company_size_bucket,annual_contract_value,product_tier,sales_segment,acquisition_channel,contract_start_date,contract_end_date,renewed_flag,discount_pct,initial_onboarding_score,is_churned
0,C100000,Company_0,Canada,North America,0,Healthcare,51-200,12999.25,Growth,MidMarket,Partner,2023-05-02,NaN,1,0.03,7.0,0
1,C100001,Company_1,UK,Europe,1,Wholesale,1-10,1799.72,Starter,SMB_Field,Inbound,2023-02-18,NaN,1,0.15,4.9,0
2,C100002,Company_2,US,North America,0,Manufacturing,1-10,1770.83,Starter,SMB_Field,Inbound,2023-11-12,2024-05-13,0,0.37,4.8,1
3,C100003,Company_3,France,Europe,1,Missing,1-10,1790.30,Starter,SMB_Field,Outbound,2023-08-06,NaN,1,0.10,8.0,0
4,C100004,Company_4,Netherlands,Europe,1,Professional Services,1-10,1552.00,Starter,SMB_Inside,Inbound,2023-09-09,2024-03-10,0,0.11,3.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,C102995,Company_2995,Canada,North America,0,Healthcare,11-50,7347.89,Growth,SMB_Inside,Inbound,2023-03-10,NaN,1,0.12,8.6,0
2996,C102996,Company_2996,UK,Europe,1,Hospitality,1000+,381926.86,Enterprise,Enterprise,Outbound,2023-05-24,NaN,1,0.15,7.0,0
2997,C102997,Company_2997,UK,Europe,1,Missing,201-1000,116504.64,Growth,Enterprise,Inbound,2023-03-10,NaN,1,0.06,7.2,0
2998,C102998,Company_2998,Spain,Europe,1,Manufacturing,1000+,461427.14,Growth,Enterprise,Outbound,2023-07-18,NaN,1,0.18,4.8,0


In [64]:
sales_cleaned[sales_cleaned["is_churned"] == 1]["customer_id"].nunique()

364